In [83]:
import os
import ipyvuetify as v
import scrapbook as sb
import yaml
import shutil

In [84]:
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)

In [85]:
config.setdefault('voila_nb_path', '.');

In [86]:
voila_nb_path = config['voila_nb_path']

In [87]:
title_bar_color = config['title_bar_color']

In [88]:
logo = {
    'jupyter': 'https://jupyter.org/assets/nav_logo.svg'
}

In [89]:
my_dashboard_name = config['title']
voila_base_url = config['server_url'] + '/voila/render/'

In [90]:
additional = []
if 'additional' in config.keys():
    additional = [dictionary for key, dictionary in config['additional'].items()]

In [91]:
lst = []
if 'app_bar_links' in config.keys():
    lst = [
        v.Btn(children=[name], flat=True, href=link, target='_blank')
        for name, link in config['app_bar_links'].items()
    ]

In [92]:
stages = ['released', 'beta']

In [93]:
structure = {}
for stage in stages:
    all_files = [f for f in os.listdir('./'+stage) if os.path.isfile('./'+stage+'/'+f)]
    files = []
    for file in all_files:
        extension = file.split('.')[-1]
        if extension == 'ipynb':
            files.append(file)
    items = []
    for i, f in enumerate(files):
        this_item = {}
        this_item['title'] = f
        this_item['description'] = 'A Jupyter Notebook.'
        this_item['link'] = voila_base_url + f
        this_item['logo'] = 'https://jupyter.org/assets/nav_logo.svg'
        this_item['fname_'] = f       
        this_item['fname'] = stage+'/'+f

        nb = sb.read_notebook('./' + stage + '/' +f)
        for key, value in nb.scraps.data_dict.items():
            this_item[key] = value
        items.append(this_item)
        structure[stage] = items

In [94]:
structure['released'].extend(additional)

In [95]:
toolbar = v.Toolbar(color=title_bar_color, dark=True,
    children=[
        v.ToolbarItems(children=[v.Img(src=logo['jupyter'], style_='height:100%')]),
        v.ToolbarTitle(children=[my_dashboard_name], color='green'),
        v.Spacer(), # Fills empty space
        v.ToolbarItems(children=lst)
    ],
    app=True
)

In [96]:
items = structure['released']

In [97]:
# this only works if voila is started in a dir; it needs to work if voila is given a target dir

moves = [
    v.Btn(icon=True, href='./homepage.ipynb', children=[v.Icon(color='grey', children=['redo'])])
    for i, details in enumerate(items)
]


def on_move_click(widget, event, data):
    fname = (structure['released'][moves.index(widget)]['fname_'])
    shutil.move(voila_nb_path+'/released/'+fname, voila_nb_path+'/beta/'+fname)
    
for button in moves:
    button.on_event('click', on_move_click)



In [98]:
cards = [
    v.Flex(ma_2=True, fluid=True, children = [
        v.Card(hover=True, #class_='mx-auto', elevation=5,
               align_center=True,
               fluid=True,
               #height='100%',
               #min_height='250px',
               min_width='300px',
               max_width='300px',
               href=details['link'],
               target='_blank',
               children=[
                   v.CardTitle(children=[
                       v.Html(tag='div', class_='headline mb-0', children=[details['title']]),
                       v.Spacer(),
                       #v.Btn(icon=True, href=details['link'], target='blank_', children=[v.Icon(color='grey', children=['launch'])])
                   ]),
                   v.CardText(children=[details['description']]),
                   v.CardActions(ma_3=True, children=[
                       moves[i],
                       v.Spacer(),
                       v.Img(src=details['logo'], max_width=80, contain=True)])
               ])
    ])
    for i, details in enumerate(items)
]

In [105]:
try:
    items = structure['beta']
except:
    items = []

dels = [
    v.Btn(icon=True, href='./homepage.ipynb', children=[v.Icon(color='grey', children=['delete'])])
    for i, details in enumerate(items)
]

def on_del_click(widget, event, data):
    fname = (items[dels.index(widget)]['fname_'])
    if not os.path.exists(voila_nb_path+'/deleted'):
        os.makedirs(voila_nb_path+'/deleted')
    shutil.move(voila_nb_path+'/beta/'+fname, voila_nb_path+'/deleted/'+fname)

for button in dels:
    button.on_event('click', on_del_click)
    
promotes = [
    v.Btn(icon=True, href='./homepage.ipynb', children=[v.Icon(color='grey', children=['undo'])])
    for i, details in enumerate(items)   
]

def on_promote_click(widget, event, data):
    fname = (structure['beta'][promotes.index(widget)]['fname_'])
    shutil.move(voila_nb_path+'/beta/'+fname, voila_nb_path+'/released/'+fname)

for button in promotes:
    button.on_event('click', on_promote_click)

In [106]:
cards_b = [
    v.Flex(ma_2=True, fluid=True, children = [
        v.Card(hover=True, #class_='mx-auto', elevation=5,
               align_center=True,
               fluid=True,
               #height='100%',
               #min_height='250px',
               min_width='300px',
               max_width='300px',
               href=details['link'],
               target='_blank',
               children=[
                   v.CardTitle(children=[
                       v.Html(tag='div', class_='headline mb-0', children=[details['title']]),
                       v.Spacer(),
                       #v.Btn(icon=True, href=details['link'], target='blank_', children=[v.Icon(color='grey', children=['launch'])])
                   ]),
                   v.CardText(children=[details['description']]),
                   v.CardActions(ma_3=True, children=[
                       promotes[i],
                       dels[i],
                       v.Spacer(),
                       v.Img(src=details['logo'], max_width=80, contain=True)])
               ])
    ])
    for i, details in enumerate(items)
]

In [107]:
tabs = v.Tabs(v_model='tab', color='grey lighten-5', fixed_tabs=True, children=[
    v.Tab(children=['released']),
    v.Tab(children=['beta']),
    v.TabItem(children=[v.Layout(ma_5=True, wrap=True, children=cards)]),
    v.TabItem(children=[v.Layout(ma_5=True, wrap=True, children=cards_b)])
])

app = v.App(
    style_="background: white",#3A1C71",
    children = [
        toolbar,
        v.Container(fluid=True, mt_3=True, children = [
            v.Layout(children=[
                v.Flex(children=[tabs])
        ])
    ])
])


In [108]:
app

App(children=[Toolbar(app=True, children=[ToolbarItems(children=[Img(src='https://jupyter.org/assets/nav_logo.…

In [61]:
    # delete = v.Btn(color='warn', children=['delete'])
    # cancel = v.Btn(children=['cancel'])
    # def close_dialog(a,b,c):
    #     dlg.v_model=False
    # cancel.on_event('click', close_dialog)

In [18]:

# items = [v.ListTile(children=[
#     v.ListTileTitle(children=[
#         f'Click me {i}'])]) 
#          for i in range(1, 5)]
# dlst = v.List(children=items)

In [19]:
??v.Footer


In [20]:
# img=v.Img(class_='ma-3', src='https://jupyter.org/assets/nav_logo.svg', max_width=80, contain=True)

In [21]:
# z = v.Card(class_='mx-auto', elevation=5,
#                align_center=True,
#                min_height='300px',
#                min_width='400px',
#                max_height='300px',
#                max_width='400px',
#                children=[
#                    v.CardTitle(children=[v.Html(tag='div', class_='headline mb-0', children=['title'])]),
#                    v.CardText(children=['description']),
#                    v.Flex(children=[v.Footer(class_='ma-3', children=[img], absolute=True)]),
#                ])

In [22]:
#dlst

In [23]:
# dlg = v.Dialog(v_model=False, width='500', children=[
#         v.Btn(icon=True, slot='activator', small=True, children=[v.Icon(color='grey', children=['delete'])]),
#         v.Card(children=[
#             v.CardTitle(children=['Delete Notebook']),
#             v.CardMedia(children=[
#                dlst
#             ]),
#             v.CardActions(children=[cancel, delete])
#         ])
# ])

In [27]:
??v.App